In [1]:
import numpy as np
import pandas as pd
# define stopwords
from nltk.corpus import stopwords
# tf-idf library
from sklearn.feature_extraction.text import TfidfVectorizer

# script files
from readfile import readTopic, readTopic2, readDocument, readDocument2
from lda import buildLDA

In [2]:
train_size = []
test_size = []
train_log_likelihood = []
train_perplexity = []
test_log_likelihood = []
test_perplexity = []
final_result = []
final_result_accuracy = []
final_result2 = []
final_result2_accuracy = []
perplexity = []
log_likelihood = []
for x in range(101, 151):
    # read training data
    num = x
    trainSet = readTopic(num)
    train_docs = []
    train_relevant = []

    for set in trainSet:
        lines = readDocument(num, set[0])
        train_docs.append(lines)
        train_relevant.append(int(set[1]))
    
    train_size.append(len(train_docs))
    
    # read testing data
    testSet = readTopic2(num)
    test_docs = []
    test_relevant = []

    for set in testSet:
        lines = readDocument2(num, set[0])
        test_docs.append(lines)
        test_relevant.append(int(set[1]))

    test_size.append(len(test_docs))    
        
    data = {'documents': train_docs, 'relevant': train_relevant}
    df = pd.DataFrame(data = data)




    # NLTK Stop words
    stop_words = stopwords.words('english')

    # build tf-idf model
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
    
    # fit and transform training data
    tfidf = vectorizer.fit_transform(train_docs)
    count_feature_names = vectorizer.get_feature_names()

    # build LDA model
    train_lda_output, train_df_document_topics, train_df, log_likelihood, perplexity = buildLDA(tfidf, train_docs, train_relevant)
    
    train_log_likelihood.append(log_likelihood)
    train_perplexity.append(perplexity)



    # build count vectorizer model
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)

    # fit training data
    tfidf = vectorizer.fit(train_docs)


    # transform testing data
    tfidf = vectorizer.transform(test_docs)
    feature_names = vectorizer.get_feature_names()
    
    # build LDA model
    test_lda_output, test_df_document_topics, test_df, log_likelihood, perplexity = buildLDA(tfidf, test_docs, test_relevant)

    test_log_likelihood.append(log_likelihood)
    test_perplexity.append(perplexity)



    # calculate the percentage of relevant of each topic
    relevant_counts = []
    irrelevant_counts = []
    percentages = []
    topics = []

    for x in range(0, len(train_df.columns)-2):
        relevant_counts.append(0)
        irrelevant_counts.append(0)

    for x in range(0, len(train_df)):
        if train_df['isRelevant'][x] == 1:
            relevant_counts[train_df['dominant_topic'][x]] += 1
        else:
            irrelevant_counts[train_df['dominant_topic'][x]] += 1

    for x in range(0, len(relevant_counts)):
        if (relevant_counts[x] > 0) and (irrelevant_counts[x] > 0):
            percentage = relevant_counts[x] / (relevant_counts[x] + irrelevant_counts[x])
            percentages.append(percentage)
            if (percentage > 0.5):
                topics.append(x)

    
    # if no topic is more than 50%, use the highest % topic
    if topics == []:
        max_value = max(relevant_counts)
        topics.append(relevant_counts.index(max_value))
    


    # generate the result of testing data
    result = []

    for x in range(0, len(test_df)):
        if test_df['dominant_topic'][x] in topics:
            result.append(1)
        else:
            result.append(0)




    test_result = pd.DataFrame()
    test_result['relevant'] = test_df['isRelevant']
    test_result['countvectorizer'] = result



    correct = 0
    for x in range(0, len(test_result)):
        if test_result['relevant'][x] == test_result['countvectorizer'][x]:
            correct += 1
    print("user: ", num, "        accuracy: ", correct/len(test_result))
    final_result.append(correct)
    final_result_accuracy.append(correct/len(test_result))
    
    
    
    # Get score(d) for each document
    topic_col = []
    for x in topics:
        topic_col.append('Topic' + str(x))

    scores = []
    for index, row in test_df.iterrows():
        score = 0
        for x in topic_col:
            score += row[x]
        scores.append(score)

    test_df['score(d)'] = scores
    df = test_df.sort_values(by=['score(d)'], ascending=False)
    df = df.head(10)
    final_result2.append((df.isRelevant.values == 1).sum())
    final_result2_accuracy.append((df.isRelevant.values == 1).sum() * 0.1)

Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -515.1693639896725
Model Perplexity:  6615.645742402023
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -5862.518816344052
Model Perplexity:  633.1676437451687
user:  101         accuracy:  0.6447140381282496
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -5799.870234487241
Model Perplexity:  8396.555199374157
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -6823.307162902565
Model Perplexity:  5272.106012527262
user:  102         accuracy:  0.4837662337662338
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -1450.3267324527264
Model Perplexity:  10398.113020143202
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -7246.05190583114
Model Perplexity:  1905.8715433338075
user: 

Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -756.328073117674
Model Perplexity:  5288.014401840882
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -6307.97110566572
Model Perplexity:  729.2844993568598
user:  125         accuracy:  0.46875
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -549.4037592408956
Model Perplexity:  3711.657857864897
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -2766.584549797621
Model Perplexity:  798.8477234743567
user:  126         accuracy:  0.37777777777777777
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -726.3725642295165
Model Perplexity:  9334.300828687543
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -2877.967754824337
Model Perplexity:  1407.5821281600797
user:  127         

Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -498.82881886435837
Model Perplexity:  5654.895657560063
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -4796.714638134622
Model Perplexity:  689.8148546737955
user:  149         accuracy:  0.1915367483296214
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -1094.8318955656655
Model Perplexity:  4699.8685555013
Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -4631.532304369635
Model Perplexity:  1028.7933569720876
user:  150         accuracy:  0.3261455525606469


In [3]:
users = []
for x in range(101, 151):
    users.append(x)
    
df = pd.DataFrame(index=users)
df['train_dataset_size'] = train_size
df['test_dataset_size'] = test_size
df['total_correct'] = final_result
df['total_accuracy'] = final_result_accuracy
df['top10_correct'] = final_result2
df['top10_accuracy'] = final_result2_accuracy
df['train_log_likelihood'] = train_log_likelihood
df['train_perplexity'] = train_perplexity
df['test_log_likelihood'] = test_log_likelihood
df['test_perplexity'] = test_perplexity
df

,train_dataset_size,test_dataset_size,total_correct,total_accuracy,top10_correct,top10_accuracy,train_log_likelihood,train_perplexity,test_log_likelihood,test_perplexity
101,23,577,372,0.644714,8,0.8,-515.169364,6615.645742,-5862.518816,633.167644
102,199,308,149,0.483766,0,0.0,-5799.870234,8396.555199,-6823.307163,5272.106013
103,64,528,184,0.348485,2,0.2,-1450.326732,10398.113020,-7246.051906,1905.871543
104,194,279,185,0.663082,0,0.0,-5661.168296,8149.672960,-6131.230195,4777.647383
105,37,258,193,0.748062,0,0.0,-741.731270,6944.454356,-2608.157388,900.330731
106,44,321,84,0.261682,1,0.1,-825.538565,5442.020841,-3785.723397,1090.669934
107,61,571,150,0.262697,0,0.0,-1410.124373,14905.760188,-7888.573990,2414.717161
108,53,386,94,0.243523,2,0.2,-1131.178560,7495.668233,-5146.129776,1419.699789
109,40,240,129,0.537500,4,0.4,-1062.097073,6777.095667,-4093.124800,1857.609283
110,91,491,53,0.107943,0,0.0,-2544.677332,8466.793201,-9718.791726,2322.317592


In [4]:
# path=r'C:\Users\sumsu\Desktop\QUT\sem2\IFN712\project\RCV1 Dataset'
# df.to_csv(path+'countvectorizer_result.csv')
df.to_csv('tfidfvectorizer_result.csv')